In [1]:
def set_gpu_memory_growth_mode(gpu_id=0):
    import tensorflow as tf
    try:
        gpus = tf.config.experimental.list_physical_devices('GPU')
        tf.config.experimental.set_memory_growth(gpus[gpu_id], True)
    except RuntimeError as e:
        print(e)

In [2]:
set_gpu_memory_growth_mode()

In [3]:
import sys
import math
from tqdm import tqdm

import data as ds
from matrix import *
from context import *
from nets.model_shuffle import *

np.set_printoptions(suppress=True)

# Context

In [4]:
config = {
    'seed' : 1234,
    'use_64bits': True,
    
    # Loss
    'loss_type': 'mse',
    
    # LEO & target parameter theta dim
    'num_latents': 16*4,
    'gen_theta_dim': 96*8 + 8,
    'num_k_shot': 5,
    'num_valid_shot': 10,
        
    # Batch & Step size
    'batch_size' : 3,
    'first_decay_steps': 20,
    'meta_lr': 1e-4,
    'latent_lr': 1e-6,
    'finetuning_lr': 1e-6,
    'num_latent_grad_steps' : 5,
    'num_finetune_grad_steps' : 5,
    'num_meta_grad_steps' : 5,
    
    'gradient_threshold': 0.2,
    'gradient_norm_threshold': 0.2,
 
    # Regularizer Term
    'dropout_rate': 0.4,
    'kl_weight': 1e-3,
    'l2_penalty_weight': 1e-8,
    'encoder_penalty_weight': 1e-9,
    'orthogonality_penalty_weight': 1e-3,
    
    'num_epochs' : 20,
    'npy_root_path': '/home/elvin/banner/mnt/ssd3/nps/',
}

ctx = Context.create(config)

# Data Provider

In [5]:
data_provider = ds.DataProvider(ctx, "01054")

In [6]:
# ASSERT 
num = len(data_provider)
tasks = data_provider[num-5]
task = tasks[ctx.batch_size-1]

assert task.tr_in1.shape == (ctx.num_k_shot, 128, 128, 3)
assert task.tr_in3.shape == (ctx.num_k_shot, 50)
assert task.tr_output.shape == (ctx.num_k_shot, 2)
assert task.val_output.shape == (ctx.num_valid_shot, 2)

In [7]:
mat = Matrix(ctx)
net = Leo.create(ctx)

# 실험 A (01054)
- 00927 tablet

In [8]:
test_task_ids = np.random.choice(len(data_provider), 6)
test_task_ids = np.random.choice(len(data_provider), 6)
print(test_task_ids)

[24 57 15 49 23 41]


In [ ]:
for i in range(8):
    net.run_with_test(50, data_provider, mat, test_task_ids, prior_task_id=0)

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


e[000] test: 453.386 |: nan |                                                           
e[000] test: 522.7919 |
e[000] test: 614.615 |
e[000] test: 424.0721 |
e[000] test: 430.0381 |
e[000] test: 359.7873 |


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


e[001] test: 41.8209 |: nan |                                                           
e[001] test: 64.3891 |
e[001] test: 36.0556 |
e[001] test: 12.4983 |
e[001] test: 27.9491 |
e[001] test: 18.7498 |
e[002] test: 11.7347 |: nan |                                                           
e[002] test: 21.6369 |
e[002] test: 5.1386 |
e[002] test: 7.9591 |
e[002] test: 11.7634 |
e[002] test: 8.0935 |
e[003] test: 6.781 |67 / 누적: 9887269.505000 |                                                                     
e[003] test: 7.8328 |
e[003] test: 6.2591 |
e[003] test: 7.9663 |
e[003] test: 7.1806 |
e[003] test: 5.3034 |
e[004] test: 5.9045 |0 / 누적: 2783.591000 |                                                               
e[004] test: 5.5709 |
e[004] test: 5.235 |
e[004] test: 6.1171 |
e[004] test: 5.308 |
e[004] test: 4.271 |
e[005] test: 5.6851 |0 / 누적: 12.325000 |                                                             
e[005] test: 6.7029 |
e[005] test: 5.487 |
e[005] test: